In [1]:
# import libraries pandas
import pandas as pd

In [2]:
# Download the phishing data
!wget http://data.phishtank.com/data/online-valid.csv

URL transformed to HTTPS due to an HSTS policy
--2023-11-25 10:02:51--  https://data.phishtank.com/data/online-valid.csv
Resolving data.phishtank.com (data.phishtank.com)... 104.16.101.75, 104.17.177.85, 2606:4700::6811:b155, ...
Connecting to data.phishtank.com (data.phishtank.com)|104.16.101.75|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn.phishtank.com/datadumps/verified_online.csv?Expires=1700886616&Signature=iWOGfmXAwvq6Cwl28wDYU5h8nRQndL1tVKoX~aLPcRmFJCcFhFwYMwD-SsUSD19K8ffjwkvu-9G3TkeN-6MbAAmllOcIWo74b9YdZRaiZCd0-GkPSfsXDt2lrzJ0QasgqVrRbBy0OMOumtKA52Orh4RdbgSUWMScpEKCYk00l4fMEI~X6UFAgT1wdYgNS-7qr3LEoePWiOvygRGYE~QQCFdaPcnUPtHT1DRv4dWyDElT4c-uyj1MpX1b4X1jS6hwkx7wY7w79C0zXS3QKNugcUcq5Owm9SxZN6V4CK0flg6NRHLmbO4bvgEQ2tdKV83rvTDWq79aiuwxGSPAfh55HA__&Key-Pair-Id=APKAILB45UG3RB4CSOJA [following]
--2023-11-25 10:02:52--  https://cdn.phishtank.com/datadumps/verified_online.csv?Expires=1700886616&Signature=iWOGfmXAwvq6Cwl28wDYU5h8nRQndL1tVKoX~a

In [3]:
# read the data
data = pd.read_csv('online-valid.csv')
data.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6557033,http://u1047531.cp.regruhosting.ru/acces-inges...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:43+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
1,6557032,http://hoysalacreations.com/wp-content/plugins...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:37+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
2,6557011,http://www.accsystemprblemhelp.site/checkpoint...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:54:31+00:00,yes,2020-05-09T21:55:38+00:00,yes,Facebook
3,6557010,http://www.accsystemprblemhelp.site/login_atte...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:53:48+00:00,yes,2020-05-09T21:54:34+00:00,yes,Facebook
4,6557009,https://firebasestorage.googleapis.com/v0/b/so...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:49:27+00:00,yes,2020-05-09T21:51:24+00:00,yes,Microsoft


In [4]:
data.shape

(14858, 8)

In [5]:
# Take 1000 random url from data column 1
data.sample(1000).iloc[:,1].to_csv('phish_starter.csv', header=False, index=False)

#### Feature Extraction

- Domain
- Ip Address
- "@" Symbol
- Length of URL
- Depth of URL
- Redirection
- HttpDomain
- URL Shortening Services
- Prefix/Suffix Separated by (-) in Domain

In [6]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [7]:
# Domain
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain

# getDomain('http://www.google.com/')

In [8]:
# Ip address
def havingIP(url):
  try:
    url = re.split(r"://",url)[1]
    ipa = re.split(r"/",url)[0]
    ipaddress.ip_address(ipa)
    ip = 1
  except:
    ip = 0
  return ip

# havingIP('http://103.182.90.12/index.html/data=good')

In [9]:
# @ symbol
def haveAtSign(url):
  if "@" in url:
    at = 1
  else:
    at = 0
  return at

# haveAtSign('http://www.google.com/')

In [10]:
# Length of URL
def getLength(url):
  if len(url) < 54:
    length = 0
  else:
    length = 1
  return length

# getLength('http://www.gooooooooooooooooooooooooooooooooooooooooooooooooooooooooooogle.com/')

In [11]:
# Depth of URL
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

# getDepth('http://www.google.com/data/hello/just/a/trial')

In [12]:
# Redirection
def redirection(url):
  redirect = ['redirect', 'redir', 'redirected', 'redirection', 'redirector', 'redirected', 'redirectors']
  for each in redirect:
    if each in url:
      return 1
  return 0

# redirection('http://www.google.com/redirect?https://evil.com')

In [13]:
# httpDomain
def httpDomain(url):
  if(re.search('^http',url)):
    return 1
  else:
    return 0
  
# httpDomain('http://www.google.com/')
  

In [14]:
# URL shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

def tinyURL(url):
  match=re.search(shortening_services,url)
  if match:
    return 1
  else:
    return 0
  
# tinyURL('http://google.com/')

In [15]:
# Prefix-Suffix
def prefixSuffix(url):
  if '-' in urlparse(url).netloc:
    return 1            # phishing
  else:
    return 0            # legitimate
  
# prefixSuffix('http://www.goog-le.com/')

In [16]:
# features extraction

def featureExtraction(url,label):
  features = []
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  features.append(label)
  return features

#### Legitimate

In [17]:
# Loading the legit data
data1 = pd.read_csv('legit_starter.csv', header=None)
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1048648/American-Snipe...
1,http://1337x.to/torrent/1110018/Blackhat-2015-...
2,http://1337x.to/torrent/1122940/Blackhat-2015-...
3,http://1337x.to/torrent/1124395/Fast-and-Furio...
4,http://1337x.to/torrent/1145504/Avengers-Age-o...


In [18]:
legit_urls = data1.sample(n=1000, random_state=12).copy()
legit_urls = legit_urls.reset_index(drop=True)
legit_urls.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,https://foursquare.com/v/%D1%81%D0%B0%D0%B2%D0...
2,http://shop-pro.jp/magazine/wp-content/themes/...
3,http://motthegioi.vn/the-gioi-xe/o-to-chat-luo...
4,http://tobogo.net/cdsb/board.php?board=dailyst...


In [19]:
legit_urls.shape

(1000, 1)

In [20]:
legit_features = []
label = 0

for i in range(0, 1000):
  url = legit_urls['URLs'][i]
  legit_features.append(featureExtraction(url,label))
  feature_names = ['Domain', 'Having_IP', 'Have_At', 'URL_Length', 'URL_Depth', 'Redirection', 'http_Domain', 'TinyURL', 'Prefix/Suffix', 'Label']
legitimate = pd.DataFrame(legit_features, columns=feature_names)
legitimate.head()

,Domain,Having_IP,Have_At,URL_Length,URL_Depth,Redirection,http_Domain,TinyURL,Prefix/Suffix,Label
0,graphicriver.net,0,0,1,1,0,1,0,0,0
1,foursquare.com,0,0,1,3,0,1,0,0,0
2,shop-pro.jp,0,0,1,6,0,1,0,1,0
3,motthegioi.vn,0,0,1,2,0,1,0,0,0
4,tobogo.net,0,0,1,2,0,1,0,0,0


In [21]:
# Storing the legitimate data
legitimate.to_csv('legitimate.csv', index=False)

#### Phishing

In [22]:
phish_urls = pd.read_csv('phish_starter.csv', header=None)
phish_urls.shape

(1000, 1)

In [23]:
phish_features = []
label = 1
for i in range(0, 1000):
  url = phish_urls.iloc[:,0][i]
  phish_features.append(featureExtraction(url,label))
  feature_names = ['Domain', 'Having_IP', 'Have_At', 'URL_Length', 'URL_Depth', 'Redirection', 'http_Domain', 'TinyURL', 'Prefix/Suffix', 'Label']
phishing = pd.DataFrame(phish_features, columns=feature_names)
phishing.head()

,Domain,Having_IP,Have_At,URL_Length,URL_Depth,Redirection,http_Domain,TinyURL,Prefix/Suffix,Label
0,turboflightpros.com,0,0,0,0,0,1,0,0,1
1,headphonestyles.buzz,0,0,0,3,0,1,0,0,1
2,secure.runescape.com-en.ru,0,0,1,2,0,1,0,1,1
3,gbgabal.com,0,0,1,4,0,1,0,0,1
4,alojamientoenobera.com.ar,0,0,0,1,0,1,0,0,1


In [24]:
# Storing the phishing data
phishing.to_csv('phishing.csv', index=False)

#### Final Dataset

In [25]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.shape

(2000, 10)

In [26]:
urldata.head()

,Domain,Having_IP,Have_At,URL_Length,URL_Depth,Redirection,http_Domain,TinyURL,Prefix/Suffix,Label
0,graphicriver.net,0,0,1,1,0,1,0,0,0
1,foursquare.com,0,0,1,3,0,1,0,0,0
2,shop-pro.jp,0,0,1,6,0,1,0,1,0
3,motthegioi.vn,0,0,1,2,0,1,0,0,0
4,tobogo.net,0,0,1,2,0,1,0,0,0


In [27]:
urldata.tail()

,Domain,Having_IP,Have_At,URL_Length,URL_Depth,Redirection,http_Domain,TinyURL,Prefix/Suffix,Label
1995,virginiawarriors.com,0,0,1,3,0,1,0,0,1
1996,vlasta.synology.me,0,1,0,1,0,1,0,0,1
1997,extrategika.com,0,0,1,4,0,1,0,0,1
1998,itcitymm.com,0,0,1,5,0,1,0,0,1
1999,cheaproomsvalencia.com,0,0,1,5,0,1,0,0,1


In [28]:
# Store the data in csv file
urldata.to_csv('urldata.csv', index=False)